# bot

In [64]:
#Your code should offer a prompt to enter a new utterance and classify this utterance, and repeat the prompt until the user exits.

def classify_utterance_1(ut):
  classified_label = "inform"
  return classified_label

def classify_utterance_2(ut):
  if 'bye' in ut:
    classified_label = 'bye'
  elif 'no' in ut:
    classified_label = 'negate'
  elif 'start over' in ut:
    classified_label = 'restart'
  else:
    classified_label = 'null'

  return classified_label

def classify_utterance_3(ut):
  #classified_label = ml_lassifier_model.predict(ut)[0]
  classified_label_id = model.predict(tfidf.transform([ut]).toarray())[0]
  classified_label = label_dict.loc[classified_label_id][0]
  return classified_label

def bot():
  finished = False
  print(f'Hi! To exit enter "bye123"')
  while not finished:
    utterance = input('>').lower()
    label = classify_utterance_3(utterance)
    print(f'label:{label}\nutterance:{utterance}')  
    if 'bye123' in utterance:
      finished = True

## ML model comparison

#exploration

In [59]:
df.describe()

,label,text
count,25501,25501
unique,15,5359
top,inform,thank you good bye
freq,10160,2565


text                                      
          count unique                      top  freq
label                                                
reqmore       5      1                     more     5
restart      14      7               start over     7
deny         27     13                    wrong    14
ack          28     20                  okay um     7
repeat       33     13                   repeat     6
hello        93     60                    hello    24
confirm     172    115  is it moderately priced    11
bye         266     41                 good bye   137
negate      435    133                       no   264
affirm     1156    166                      yes   803
null       1612    249                    noise   408
reqalts    1747    437   is there anything else   302
thankyou   3259     81       thank you good bye  2565
request    6494    961             phone number   914
inform    10160   3062              i dont care   613